In [26]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## General Stuff
################################################################################
from timeUtils import clock, elapsed
from listUtils import getFlatList
from time import sleep
from pandas import DataFrame, Series
from ioUtils import getFile, saveFile
from searchUtils import findDirs, findNearest, findAll
from fileUtils import getDirBasics, getBaseFilename
from unicodedata import normalize
from fsUtils import moveDir, setDir, mkDir, isDir, removeDir
from pandasUtils import getRowDataByColValue, getRowData
import operator
from glob import glob
from os.path import join
from collections import Counter


################################################################################
## Mp3 Stuff
################################################################################
from mp3id import mp3ID


################################################################################
## Database Stuff
################################################################################
from discogsBase import discogs
from mainDB import getAlbumTypes
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import getArtistAlbumsDB, discConv


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap
from musicBase import myMusicBase
from musicMatch import myMatchedMusic


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-24 20:17:25.039697


# My Music Database Map

In [2]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()

Current Time is Fri Apr 24, 2020 20:06:07 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Fri Apr 24, 2020 20:06:36 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 28 seconds.
Summary Statistics For DB: Discogs
    Found 718816 ID -> Name entries
    Found 660527 Name -> ID entries
    Found 0 Albums
Current Time is Fri Apr 24, 2020 20:06:37 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Fri Apr 24, 2020 20:06:39 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
==================================

# Music Base

In [3]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase()
mmb.findArtistAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']
Current Time is Fri Apr 24, 2020 20:06:42 for Finding All Artist Albums
Current Time is Fri Apr 24, 2020 20:07:21 for Done with Finding All Artist Albums
Process [Done with Finding All Artist Albums] took 39 seconds.


# Create My Matched Music Artists

In [27]:
mmm = myMatchedMusic(mdb)

My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']


## Get Artist Status and Find Unknowns

In [9]:
mmm.getArtistStatus()
mmm.setMusicBase(mmb)
mmm.getUnknownArtists()

Current Time is Fri Apr 24, 2020 20:10:26 for Matching All Music Artists
Current Time is Fri Apr 24, 2020 20:10:27 for Done with Matching All Music Artists
Process [Done with Matching All Music Artists] took 886.0 millseconds.
Found 1 unknown artists
Found 0 total artists


{"Killin' Baudelaire": ["/Volumes/Biggy/Matched/K/Killin' Baudelaire"]}

***
***

# Match Albums From My Matched Artists

In [10]:
dR = 0.1
rC = 0.75

mmm.matchMyMusicAlbums(db="AllMusic", albumType=1, ratioCut=rC, maxCut=rC+dR)

Current Time is Fri Apr 24, 2020 20:10:27 for Checking for Albums Matches Against AllMusic DB
Artist                                  Database       Album Name                                    --> Matched Album
Alison Moyet                            AllMusic       Singles                                       --> {'Name': 'Singles/Alf', 'Code': '0001089227', 'MediaType': 'Comp'}
Artist ID [0000691561] is not found in Albums DB [AllMusic]
Cornelius                               AllMusic       Cornelius FM                                  --> {'Name': 'CM Cornelius Mix', 'Code': '0000602808', 'MediaType': 'Single/EP'}
DJ Shadow                               AllMusic       Enuff EP                                      --> {'Name': 'Enuff', 'Code': '0001689967', 'MediaType': 'Single/EP'}
DJ Shadow                               AllMusic       What Does Your Soul Look Like (Peshay-DJ Die Remixes) EP --> {'Name': 'What Does Your Soul Look Like Remixes', 'Code': '0001026639', 'MediaType': '

In [28]:
mmm.moveMyMatchedMusicAlbums(show=False)

Found 1 music <-> discogs albums maps
==> DJ Shadow
	Moving /Volumes/Biggy/Matched/D/DJ Shadow/Enuff EP  --->  /Volumes/Biggy/Matched/D/DJ Shadow/Match/Single-EP/Enuff :: [AM-0001689967]


NameError: name 'moveDir' is not defined

***
***

# UnMatch Artist

In [ ]:

def unMatchArtist(artistName):
    from os.path import join
    from fsUtils import removeDir, isFile, setFile, removeFile

    for musicDir in getMatchedDirs():
        dirval        = join(musicDir, getPrimeDirectory(artistName), artistName)
        if not isDir(dirval):
            continue
            
        matchedDir    = setDir(dirval, "Match")
        mediaTypeDirs = findDirs(matchedDir)
        for mediaTypeDir in mediaTypeDirs:
            for matchDir in findDirs(mediaTypeDir):
                albumName = getUnMatchedDirName(getDirBasics(matchDir)[-1], mediaTypeDir)

                srcDir = matchDir
                dstDir = setDir(dirval, albumName)
                if isDir(dstDir):
                    i = 0
                    while not isDir(dstDir):
                        dstDir = "{0} [Fix-{1}]".format(setDir(dirval, albumName), i)
                        i += 1

                moveDir(srcDir, dstDir, debug=True)

            if isDir(mediaTypeDir):
                DS_Store = setFile(mediaTypeDir, ".DS_Store")
                if isFile(DS_Store):
                    removeFile(DS_Store, debug=True)
                removeDir(mediaTypeDir, debug=True)

        if isDir(matchedDir):
            DS_Store = setFile(matchedDir, ".DS_Store")
            if isFile(DS_Store):
                removeFile(DS_Store, debug=True)
            removeDir(matchedDir, debug=True)
            

In [ ]:
unMatchArtist("The J. Geils Band")

In [ ]:
######################################################################
#### Loop Over Prime Directories
######################################################################
for primeDir in getPrimeDirectories():
    artistPrimeDirMap = getArtistPrimeDirMap(primeDir)

    ######################################################################
    #### Loop Over Artist Name <-> Prime Map Items
    ######################################################################
    for artistName, artistPrimeDirs in artistPrimeDirMap.items():
        unMatchArtist(artistName)
